In [1]:
# try some preprocessing with mne .....
import mne
import autoreject
import numpy as np
import pandas as pd
import scipy
import re
import random

# some hyperparameter type things :)
sr = 200
t = 200
ch = 19
affix = '_mne_clean'

In [12]:
mneList = []
montage = mne.channels.make_standard_montage('standard_1020')

for subj in [('A_405', ''), ('A_408', '_HFREQ'), ('B_110', ''), ('B_309', '_HFREQ'), ('B_311', '_HFREQ'), ('B_316', ''), ('C_204', ''), ('C_429', '_HFREQ'), ('E_321', '_HFREQ'), ('E_415', '_HFREQ'), ('E_429', '_HFREQ'), ('F_027', ''), ('F_209', ''), ('F_210', '_HFREQ'), ('G_413', '_HFREQ'), ('G_428', '_HFREQ'), ('H_804', '_HFREQ'), ('I_719', '_HFREQ'), ('I_723', '_HFREQ')]:
    # print(subj[0])\n",
    sr = 1000 if subj[1] == '_HFREQ' else 200
    mat = scipy.io.loadmat(f'preprocessing/raw_data/{subj[0]}{subj[1]}.mat')
    data = mat['o'][0][0][5].T
    data = np.delete(data, [21], axis = 0)
    ch = 19
    
    if sr == 1000: data = scipy.signal.resample(data, int(data.shape[1]/5), axis = 1)
    events = pd.read_csv(f'preprocessing/{subj[0]}_events.txt', sep="\t").drop('number', axis=1)
    events = events[['latency', 'urevent', 'type']].to_numpy()
    events = mne.pick_events(events, include=[1, 2, 3, 4, 5]) # should replace this with a numpy function lol\n",
    events = np.array(events, dtype=int)
    
    # mne preprocessing
    for event in events:
        if sr==1000: event[0] = int(event[0]/5)
        
    sr = 200
    raw = mne.io.RawArray(data, mne.create_info(ch_names = ['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'A1', 'A2', 'F7', 'F8', 'T3', 'T4', 'T5', 'T6', 'Fz', 'Cz', 'Pz'], sfreq = sr, ch_types='eeg'))
    # raw.set_montage(montage)
    raw.filter(l_freq=0.5, h_freq=60)
    raw.set_eeg_reference(ref_channels='average')
    epochs = mne.Epochs(raw, events, event_id=[1, 2, 3, 4, 5], tmin=-0.2, tmax=0.8, baseline=(None,0), on_missing='warn', preload=True)
    # ar = autoreject.AutoReject(n_interpolate=[1, 2, 3, 4], random_state=11,n_jobs=1, verbose=True)
    # ar.fit(epochs[:50])
    # file = ar.transform(epochs)

    mneList.append((subj[0], epochs))
    
mneDict = dict(mneList)
subjList = ['A_405', 'A_408', 'B_110', 'B_309', 'B_311', 'B_316', 'C_204', 'C_429', 'E_321', 'E_415', 'E_429', 'F_027', 'F_209', 'F_210', 'G_413', 'G_428', 'H_804', 'I_719', 'I_723']
fileList = []
    
for letter in ['A', 'B', 'C', 'E', 'F', 'G', 'H', 'I']:
    r = re.compile(f'{letter}_d*')
    subjects = list(filter(r.match, subjList))
    fileName = f'subj_{letter}'
    epochs = mne.concatenate_epochs([mneDict[subj] for subj in subjects])
    fileList.append((fileName, epochs))

mneList = []
for subj, file in fileList:
    features = file.get_data()
    features = np.delete(features, [10, 11], axis = 1)
    features = features[:, :, :-1]
    labels = file.events[:, -1] - 1
    mneList.append((subj, features, labels))
    np.save(f'pickles/{subj}_features{affix}', features)
    np.save(f'pickles/{subj}_labels{affix}', labels)
    
# fileList is subj, file.
# mneList can be subj, features, labels?

Creating RawArray with float64 data, n_channels=21, n_times=718600
    Range : 0 ... 718599 =      0.000 ...  3592.995 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 60 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 60.00 Hz
- Upper transition bandwidth: 15.00 Hz (-6 dB cutoff frequency: 67.50 Hz)
- Filter length: 1321 samples (6.605 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s


EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.


[Parallel(n_jobs=1)]: Done  21 out of  21 | elapsed:    0.6s finished


Not setting metadata
959 matching events found
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 959 events and 201 original time points ...
0 bad epochs dropped
Creating RawArray with float64 data, n_channels=21, n_times=719000
    Range : 0 ... 718999 =      0.000 ...  3594.995 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 60 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 60.00 Hz
- Upper transition bandwidth: 15.00 Hz (-6 dB cutoff frequency: 67.50 Hz)
- Filter length: 1321 samples (6.605 s)



...

0 bad epochs dropped
Creating RawArray with float64 data, n_channels=21, n_times=718200
    Range : 0 ... 718199 =      0.000 ...  3590.995 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 60 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 60.00 Hz
- Upper transition bandwidth: 15.00 Hz (-6 dB cutoff frequency: 67.50 Hz)
- Filter length: 1321 samples (6.605 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s


EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Not setting metadata
958 matching events found
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 958 events and 201 original time points ...


[Parallel(n_jobs=1)]: Done  21 out of  21 | elapsed:    0.5s finished


0 bad epochs dropped
Not setting metadata
1826 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
3811 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
1899 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
2848 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
2874 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
1901 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
922 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
1917 matching events found
Applying baseline correction (mode: mean)


In [3]:
mneList = []
for letter in ['A', 'B', 'C', 'E', 'F', 'G', 'H', 'I']:
    features = np.load(f'pickles/subj_{letter}_features{affix}.npy')
    labels = np.load(f'pickles/subj_{letter}_labels{affix}.npy')
    mneList.append((f'subj_{letter}', features, labels))

In [4]:
rawList = []
affix = '_raw'
for letter in ['A', 'B', 'C', 'E', 'F', 'G', 'H', 'I']:
    features = np.load(f'pickles/subj_{letter}_features{affix}.npy')
    labels = np.load(f'pickles/subj_{letter}_labels{affix}.npy')
    rawList.append((f'subj_{letter}', features, labels))

In [22]:
# pseudosampling

window = 100 # window size
strides = [25, 10, 10, 5, 5, 10, 10, 25] # step size (but actually it's better to make it BELL CURVE OMGGGGGGG (for overlap % maybe then...))
# strides = [50, 25, 25, 25, 25, 50] #this is if samples are 400 total 
pseudosampled_features = {}
pseudosampled_labels = {}

for idx, (subj, features, labels) in enumerate(mneList):
    _features = np.empty((features.shape[0], features.shape[1]* len(strides) + 1, window))
    for trial_i in range(len(labels)):
        start = 0
        for s in strides:
            end = start + window
            for channel_i in range(ch):
                #print(features[trial_i][channel_i].shape, start, end, window)
                _features[trial_i][channel_i] = features[trial_i][channel_i][start:end]
            start += s
    pseudosampled_features[subj] = _features
    pseudosampled_labels[subj] = labels
    fileList[idx] = (subj, _features, labels)

In [32]:
from scipy.signal import stft

STFT_features = {}
STFT_labels = {}

for subj, features, labels in mneList:
    _features = []
    for trial_i in range(labels.shape[0]): #for each event
        features_flat = []
        for channel_i in range(19): #srry hardcoding this cuz lazy
            sample = features[trial_i][channel_i]
            f, t, Zxx = stft(sample, fs=100, window='triang', boundary= None)
            _features.append(Zxx)
    print(len(_features), features.shape)
    # features.to_csv(f'{subj}/fft/features.csv', index=False)
    # labels.to_csv(f'{subj}/fft/labels.csv', index=False)
    STFT_features[subj] = _features
    STFT_labels[subj] = labels

/net/home/store/home/cchen4/5F_EEG_DATA/.venv/lib/python3.10/site-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 200, using nperseg = 200
  warnings.warn('nperseg = {0:d} is greater than input length '


34694 (1826, 19, 200)
72409 (3811, 19, 200)
36081 (1899, 19, 200)
54112 (2848, 19, 200)
54606 (2874, 19, 200)
36119 (1901, 19, 200)
17518 (922, 19, 200)
36423 (1917, 19, 200)


In [14]:
# SVM TIME WOOO
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics

avg_acc = 0

for subj, features, labels in mneList:
    # features = fft_features[subj][:, 0:10]
    # labels = fft_labels[subj]
    X_train, X_test, y_train, y_test = train_test_split(np.reshape(features, (features.shape[0], -1)), labels, test_size= 0.3) # 70% training 30% test
    
    classifier = BaggingClassifier(svm.SVC(kernel='rbf'), n_estimators=20) # radial basis kernel; er i think they used gamma=0.2 ngl
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    avg_acc += metrics.accuracy_score(y_test, y_pred)

    print(f"Accuracy {subj}:",metrics.accuracy_score(y_test, y_pred))

print("Average accuracy: ", avg_acc/len(fileList))

Accuracy subj_A: 0.35583941605839414
Accuracy subj_B: 0.44667832167832167
Accuracy subj_C: 0.4789473684210526
Accuracy subj_E: 0.5625730994152047
Accuracy subj_F: 0.4727694090382387
Accuracy subj_G: 0.4168126094570928
Accuracy subj_H: 0.30685920577617326
Accuracy subj_I: 0.4444444444444444
Average accuracy:  0.4356154842861153


In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn import metrics

avg_acc = 0
for subj, features, labels in mneList:
    X_train, X_test, y_train, y_test = train_test_split(np.reshape(features, (features.shape[0], -1)), labels, test_size= 0.3) # 70% training 30% test
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)

    num_rounds = 10
    param = {'max_depth': 3, 'eta': 1, 'objective': 'multi:softmax', 'num_class': 5}
    watchlist = [(dtest, 'eval'), (dtrain, 'train')]

    bst = xgb.train(param, dtrain, num_boost_round=num_rounds, evals=watchlist)
    y_pred = bst.predict(dtest)
    avg_acc += metrics.accuracy_score(y_test, y_pred)

    print(f"Accuracy {subj}:",metrics.accuracy_score(y_test, y_pred))

print(f'Avg Accuracy: {avg_acc/len(fileList)}')


[0]	eval-mlogloss:1.56275	train-mlogloss:1.30989
[1]	eval-mlogloss:1.60651	train-mlogloss:1.12568
[2]	eval-mlogloss:1.62107	train-mlogloss:0.98113
[3]	eval-mlogloss:1.66023	train-mlogloss:0.85128
[4]	eval-mlogloss:1.69275	train-mlogloss:0.75137
[5]	eval-mlogloss:1.72510	train-mlogloss:0.66750
[6]	eval-mlogloss:1.73434	train-mlogloss:0.58659
[7]	eval-mlogloss:1.75414	train-mlogloss:0.51112
[8]	eval-mlogloss:1.77201	train-mlogloss:0.44886
[9]	eval-mlogloss:1.80113	train-mlogloss:0.39457
Accuracy subj_A: 0.3083941605839416
[0]	eval-mlogloss:1.55182	train-mlogloss:1.37267
[1]	eval-mlogloss:1.50617	train-mlogloss:1.23053
[2]	eval-mlogloss:1.48458	train-mlogloss:1.10939
[3]	eval-mlogloss:1.46652	train-mlogloss:1.01800
[4]	eval-mlogloss:1.46139	train-mlogloss:0.93324
[5]	eval-mlogloss:1.46257	train-mlogloss:0.85779
[6]	eval-mlogloss:1.45599	train-mlogloss:0.78564
[7]	eval-mlogloss:1.46100	train-mlogloss:0.72110
[8]	eval-mlogloss:1.46749	train-mlogloss:0.66733
[9]	eval-mlogloss:1.47540	train-m

In [7]:
# modified 2D CNN :D

import torch
import torch.nn as nn
import torch.optim as optim

class ConvNet_2D(nn.Module):
    def __init__(self):
        super(ConvNet_2D, self).__init__()
        ### FILL IN ### [10 POINTS]

        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 16, (7, 7)),
            nn.ReLU(), 
            nn.MaxPool2d(2)
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 32, (5, 7)),
            #nn.Conv2d(16, 32, (3, 7)),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )


        self.hidden = nn.Sequential(
            nn.Flatten(),
            # nn.Linear(121728, 256),
            nn.Linear(1440, 256),
            nn.ReLU(),
            nn.Dropout(0.4), #
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(128, 5),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        ### FILL IN ### [5 POINTS]
        x = self.conv1(x)
        x = self.conv2(x)
        # x = self.conv3(x)
        x = self.hidden(x)
        return x


In [10]:
# run CNN
from sklearn.model_selection import train_test_split
# train neural net
torch.cuda.set_device(4)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# torch.manual_seed(42)
# np.random.seed(42)
device = 'cpu'
accuracies = []

for subj, features, labels in rawList:
    # features = pseudosampled_features[subj]
    # labels = pseudosampled_labels[subj]
    
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size= 0.2) # 70% training 30% test
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size= 0.2) # 70% training 30% test
    conv_net = ConvNet_2D().to(device)
    criterion = nn.CrossEntropyLoss()
    conv_net_optimizer = torch.optim.Adam(conv_net.parameters(), lr=0.001)
    # Train the neural network
    train_acc = 0
    for epoch in range(100):
        # print("epoch = ",epoch)
        conv_net.train()
        data = X_train
        targets = y_train
        data = torch.tensor(data.reshape(int(data.shape[0]), 1, data.shape[1], data.shape[2]), dtype=torch.float32) # reshape # of trial, 1 channel, # of samples
        data = data.to(device)
        targets = torch.tensor(targets, dtype=torch.int64)
        targets = targets.to(device)

        # Forward pass
        conv_net_predictions = conv_net(data)
        conv_net_loss = criterion(conv_net_predictions, targets)

        total, correct = 0, 0
        _, predicted = torch.max(conv_net_predictions.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()
        
        # Backward pass
        conv_net_optimizer.zero_grad()
        conv_net_loss.backward()
        conv_net_optimizer.step()
        train_acc = correct/total

    print(f"train accuracy {subj}= ", train_acc) 

    # Evaluate the neural network
    conv_net.eval()
    correct, total = 0, 0
    with torch.no_grad():
        data = X_test
        targets = y_test
        data = torch.tensor(data.reshape(int(data.shape[0]), 1, data.shape[1], data.shape[2]), dtype=torch.float32) # reshape # of trial, 1 channel, # of samples
        data = data.to(device)
        targets = torch.tensor(targets, dtype=torch.int64)
        targets = targets.to(device)
        conv_net_predictions = conv_net(data)
        _, predicted = torch.max(conv_net_predictions.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()
    print(f"test accuracy {subj}= ",correct/total, '****************************')
    accuracies.append(correct/total)

print(f'avg accuracy = {np.mean(accuracies)}')


train accuracy subj_A=  0.8416095890410958
test accuracy subj_A=  0.3879781420765027 ****************************


train accuracy subj_B=  0.8297785069729287
test accuracy subj_B=  0.4325032765399738 ****************************
train accuracy subj_C=  0.9333333333333333
test accuracy subj_C=  0.5447368421052632 ****************************
train accuracy subj_E=  0.9297475301866082
test accuracy subj_E=  0.5771929824561404 ****************************
train accuracy subj_F=  0.9026644915715063
test accuracy subj_F=  0.4017391304347826 ****************************
train accuracy subj_G=  0.90625
test accuracy subj_G=  0.49343832020997375 ****************************
train accuracy subj_H=  0.9303904923599321
test accuracy subj_H=  0.2810810810810811 ****************************
train accuracy subj_I=  0.8866231647634584
test accuracy subj_I=  0.5208333333333334 ****************************
avg accuracy = 0.45493788852963135


In [ ]:
# ConvNet_Finger (both models and helpers)

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

class ConvNet_Finger(nn.Module):
    def __init__(self):
        super(ConvNet_Finger, self).__init__()
        ### FILL IN ### [10 POINTS]

        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 8, (7, 3)),
            nn.ReLU(), 
            #nn.MaxPool2d(2)
        )
        
        self.conv2 = nn.Sequential(
            nn.Conv2d(8, 16, (5, 3)),
            nn.Conv2d(16, 32, (3, 3)),
            nn.ReLU(),
            #nn.MaxPool2d(2)
        )

        self.hidden = nn.Sequential(
            nn.Flatten(),
            # nn.Linear(121728, 256),
            nn.Linear(43456, 256),
            nn.ReLU(),
            nn.Dropout(0.4), # all features norefer
            # nn.Linear(1408, 256), #pseudosampled featrues .
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(128, 2),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        ### FILL IN ### [5 POINTS]
        x = self.conv1(x)
        x = self.conv2(x)
        # x = self.conv3(x)
        x = self.hidden(x)
        return x
 
# final net

class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        ### FILL IN ### [10 POINTS]

        self.hidden = nn.Sequential(
            nn.Flatten(),
            # nn.Linear(121728, 256),
            nn.Linear(43456, 256),
            nn.ReLU(),
            nn.Dropout(0.4), # all features norefer
            # nn.Linear(1408, 256), #pseudosampled featrues .
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(128, 5),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        nn.Sequential(
            nn.Linear(5, 16),
            nn.Linear(16, 32),
            nn.Linear(32, 5)
        )
        return x
    
# help :P

def predict(y_train):
    data = y_train
    pred = []
    with torch.no_grad():
        model.eval()
        data = data.to(device)
        predictions = model(data)
        _, predictions = predictions.max(1)
    return predictions

def evaluate(predictions, targets):
    n_samples, n_correct = 0, 0
    # print(targets.shape, predictions.shape)
    # print(type(targets), targets.shape, type(predictions), len(predictions))
    n_samples += targets.size(0)
    n_correct += (predictions == targets).sum().item()
    acc = n_correct / n_samples
    print(f'accuracy = {acc}')
    return acc

def binary_confusion(y_pred, target):
    # matrix represents [pred 0 targ 0, pred 0 targ 1] [pred 1 targ 0, pred 1 targ 1]
    matrix = [[0, 0],[0, 0]]
    for i in range(y_pred.shape[0]):
        #zero stands for not thumb
        if y_pred[i] == 0 and target[i] == 0:
            matrix[0][0] += 1
        elif y_pred[i] == 0 and target[i] == 1:
            matrix[0][1] += 1
        elif y_pred[i] == 1 and target[i] == 0:
            matrix[1][0] += 1
        elif y_pred[i] == 1 and target[i] == 1:
            matrix[1][1] += 1
    matrix = matrix/np.sum(matrix)
    return matrix

In [ ]:
# run ensemble :P
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings("ignore")


batch_size = 32
dropout = 0.2
n_epochs = 30
learning_rate = 1e-3
weight_decay = 1e-6
input_shape = (-1, 1, ch, t)

torch.cuda.set_device(5)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
sm = SMOTE()

# train neural net
accuracies = []
for subj, features, labels in mneList:
    new_features = []
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size= 0.3) # 70% training 30% test
    for finger in range(5):
        # print(X_train.shape, y_train.shape)
        y_train_ = np.where(y_train == finger, 1, 0)
        y_test_ = np.where(y_test == finger, 1, 0)
        X_res = X_train.reshape((X_train.shape[0], -1))
        X_res, y_res = sm.fit_resample(X_res, y_train_)
        X_res = X_res.reshape((X_res.shape[0], ch, -1))

        model = ConvNet_Finger().to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        # Train the neural network

        model.train()
        data = X_res
        targets = y_res
        data = torch.tensor(data.reshape(int(data.shape[0]), 1, data.shape[1], data.shape[2]), dtype=torch.float32) # reshape # of trial, 1 channel, # of samples
        data = data.to(device)
        targets = torch.tensor(targets, dtype=torch.int64)
        targets = targets.to(device)

        for epoch in range(n_epochs):
            # print("epoch = ",epoch)
            model.train()

            # Forward pass
            predictions = model(data)
            loss = criterion(predictions, targets)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        torch.save(model.state_dict(), f'pickles/models/{subj}_finger_{finger}.pt')
        
        # model.load_state_dict(torch.load(f'pickles/models/{subj}_finger_{finger}.pt'))
        
        pred = predict(torch.as_tensor(X_train.reshape(X_train.shape[0], 1, ch, t), dtype=torch.float32).to(device))
        new_features.append(pred*finger+1)
    new_features = torch.stack(new_features)
    features = torch.transpose(torch.tensor(new_features, dtype=torch.float32), 0, 1).to(device).requires_grad_(True)
    # print(features.shape, features)
    
    # final NeuralNet
    model = NeuralNet().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Train the neural network
    model.train()
    data = features
    targets = torch.tensor(y_train, dtype=torch.int64).to(device)

    for epoch in range(n_epochs):
        # print("epoch = ",epoch)
        model.train()

        # Forward pass
        predictions = model(data)
        loss = criterion(predictions, targets)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    torch.save(model.state_dict(), f'pickles/models/{subj}_final.pt')

    print(f'trained {subj} {finger}')
    new_features = []

    for finger in range(5):
        model = ConvNet_Finger().to(device)
        criterion = nn.CrossEntropyLoss
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        model.load_state_dict(torch.load(f'pickles/models/{subj}_finger_{finger}.pt'))

        y_res = np.where(y_test == finger, 1, 0)
        y_res = torch.tensor(y_res, dtype=torch.int64).to(device)
        pred = predict(torch.tensor(X_test.reshape(input_shape), dtype=torch.float32).to(device))
        print(f'accuracy {subj} finger {finger} = {evaluate(pred, y_res)} {binary_confusion(pred, y_res)})')
        new_features.append(pred*finger)


    new_features = torch.stack(new_features)
    features = torch.transpose(torch.tensor(new_features, dtype=torch.float32), 0, 1).to(device).requires_grad_(True)
    
    model = NeuralNet().to(device)
    model.load_state_dict(torch.load(f'pickles/models/{subj}_final.pt'))
    acc = evaluate(predict(features), torch.tensor(y_test, dtype=torch.int64).to(device))
    print(f'FINAL accuracy {subj} = {acc} ***************************************')
    accuracies.append(acc)

print(f'avg accuracy = {np.mean(accuracies)}')

trained subj_A 4
accuracy = 0.75
accuracy subj_A finger 0 = 0.75 [[0.67335766 0.13686131]
 [0.11313869 0.07664234]])
accuracy = 0.7992700729927007
accuracy subj_A finger 1 = 0.7992700729927007 [[0.7810219  0.15693431]
 [0.04379562 0.01824818]])
accuracy = 0.7408759124087592
accuracy subj_A finger 2 = 0.7408759124087592 [[0.69160584 0.15693431]
 [0.10218978 0.04927007]])
accuracy = 0.7335766423357665
accuracy subj_A finger 3 = 0.7335766423357665 [[0.70255474 0.17518248]
 [0.09124088 0.0310219 ]])
accuracy = 0.7883211678832117
accuracy subj_A finger 4 = 0.7883211678832117 [[0.73540146 0.1459854 ]
 [0.06569343 0.05291971]])
accuracy = 0.2791970802919708
FINAL accuracy subj_A = 0.2791970802919708 ***************************************
trained subj_B 4
accuracy = 0.8094405594405595
accuracy subj_B finger 0 = 0.8094405594405595 [[0.7027972  0.11101399]
 [0.07954545 0.10664336]])
accuracy = 0.7788461538461539
accuracy subj_B finger 1 = 0.7788461538461539 [[0.72115385 0.1284965 ]
 [0.09265734

KeyboardInterrupt: 

In [ ]:
# fft features

from scipy import fftpack
import matplotlib.pyplot as plt

sample_freqs = fftpack.fftfreq(t) # since all samples are the same, sample freqs are the same

fft_features = {}
fft_labels = {}

for subj, features, labels in mneList:
    # file.event_id = {'thumb':1, 'index':2, 'middle':3, 'ring':4, 'pinkie':5}
    # conditions = ['thumb']
    fft_features[subj] = np.zeros((labels.shape[0], ch, int(t/2)))
    for trial_i in range(labels.shape[0]): #for each event
        fft = np.zeros((ch, int(t/2)))
        for channel_i in range(ch):
            sample = features[trial_i][channel_i]
            power = sig_fft = fftpack.fft(sample)
            power = np.abs(sig_fft)
            sample_freq = fftpack.fftfreq(sample.size)
            fft[channel_i] = power[0:int(power.size/2)]
        fft_features[subj][trial_i] = fft
    fft_labels[subj] = labels
    features = fft_features[subj]

In [ ]:
def wpd(X):
    coeffs = pywt.WaveletPacket(X, 'db4', mode='symmetric', maxlevel=8)
    return coeffs

def feature_bands(x):
    bands = np.empty((8, x.shape[0], x.shape[1], 30))
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            c = wpd(x[i, j, :])
            pos = [node.path for node in c.get_level(5, 'natural')]
            for b in range(8):
                data = c[pos[b]].data
                if len(data) < 30:
                    data = np.pad(data, (0, 30-len(data)))
                elif len(data) > 30:
                    data = data[:30]
                bands[b, i, j, :] = data
    return bands

In [ ]:
# marios processing thing
from mne.decoding import CSP
from sklearn.preprocessing import StandardScaler
import pywt

cspList = []
for subj, file in fileList:
    wpd_data = feature_bands(file.get_data())
    csp = [CSP(n_components=5, reg=None, log=True, norm_trace=False) for _ in range(8)]
    ss = StandardScaler()
    features = ss.fit_transform(np.concatenate(tuple(csp[x].fit_transform(wpd_data[x], file.events[:, -1]) for x in range(8)), axis=-1))
    cspList.append((subj, features, file.events[:, -1] -1))
                                

Computing rank from data with rank=None
    Using tolerance 58 (2.2e-16 eps * 21 dim * 1.2e+16  max singular value)
    Estimated rank (mag): 20
    MAG: rank 20 computed from 21 data channels with 0 projectors
    Setting small MAG eigenvalues to zero (without PCA)
Reducing data rank from 21 -> 20
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 52 (2.2e-16 eps * 21 dim * 1.1e+16  max singular value)
    Estimated rank (mag): 20
    MAG: rank 20 computed from 21 data channels with 0 projectors
    Setting small MAG eigenvalues to zero (without PCA)
Reducing data rank from 21 -> 20
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 50 (2.2e-16 eps * 21 dim * 1.1e+16  max singular value)
    Estimated rank (mag): 20
    MAG: rank 20 computed from 21 data channels with 0 projectors
    Setting small MAG eigenvalues to zero (without PCA)
Reducing data rank from 21 -> 20
Estimating c

LinAlgError: SVD did not converge

In [ ]:
# get CSP features :D
from mne.decoding import CSP

csp_features = {}
csp_labels = {}

for subj, features, labels in fileList:
    csp = CSP(n_components=5, reg=None, log=True)
    labels = raw_labels[subj]

    csp_features[subj] = csp.fit_transform(features, labels)
    csp_labels[subj] = labels

    csp.plot_filters(file.info, title='CSP patterns for %s' % subj)    
